# Exploring cities around Europe 

## 1. Introduction

### 1.1 Description 

As everybody else, at some point, I started thinking about cities where I would enjoy living in. In my case, those thoughts came to my mind after I finished my Computer Engineering studies in Alicante (a beautiful and sunny city in the south-east of Spain). Thanks to the many posibilities for exchanges and internships that the University of Alicante gave me, I have gained some ideas about what is important to think about when you are planning about moving to a new city. 

In this project, we will take into account only countries in the European Union. The EU is a unique economic and political union between 28 EU countries that together cover much of the continent. It was created in 1958 and since then It has eliminated borders between the belonged countries, facilitating live, work and travelling abroad in Europe. Also, It created the Euro, a single currency which it mission is to stabilize the economy and help to get equality between the countries. 

But, of course, all these cities are very different between them, total population, culture, yearly sunny hours, language, average salary and working hours are some of the factors that completely change how is to live in a city. We will cluster by how similar they are in comparison to each other.

### 1.2 Data descrition

Classify a city can be very complicated and forces to get a lot of data from different sources:

- We will get the cities with it population and countries from wikipedia: 
https://en.wikipedia.org/wiki/List_of_cities_in_the_European_Union_by_population_within_city_limits

- From Foursquare API, we will extract relevant information about venues of each city.

- Using Meteoblue, we will get the coordinates from each city.

## 2. Data extraction

To start, we will extract all the cities from the wikipedia link in the previous paragraph.

First, we import all the needed libraries and download the data of the link:

In [39]:
from requests import get
import pandas as pd
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
import numpy as np

Parser the html, structure and clean the data to extract name of the city, country and population:

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_cities_in_the_European_Union_by_population_within_city_limits'
response = get(url)
html_soup = BeautifulSoup(response.text, 'html.parser')
table = html_soup.find('table', {'class': 'wikitable sortable'})
table_rows = table.find_all('tr')
table_columns = table.find_all('th')

c = []
for th in table_columns:
    text = th.text.replace('\n', '')
    c.append(text)
c = c[0:3]

r = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    
    #Handle if there is an empty row, if not, exception will be thrown
    if(row == []):
        continue
        
    #format data correctly
    row[0] = row[0].split('[')[0]
    row[1] = row[1].strip('\xa0').strip('\n')
    row[2] = row[2].split('♠')[1].strip('\n')
    row = row[0:3]
    
    r.append(row)
df = pd.DataFrame(r, columns=c)

#change name of population column.
df=df.rename(columns = {'Officialpopulation':'Population'})

Display the 5 cities in EU with more population:

In [3]:
df.head()

,City,Country,Population
0,London,United Kingdom,"8,825,001"
1,Berlin,Germany,"3,723,914"
2,Madrid,Spain,"3,223,334"
3,Rome,Italy,"2,863,970"
4,Paris,France,"2,140,526"


Let's see the shape of the dataframe

In [4]:
df.shape

(110, 3)

Extract from meteoblue API the coordinates for each city

In [5]:
locationCities = []
for city in df['City']:
    # There is a problem with Cordoba, there is a bigger city in south america (extract adding country for next version)
    if(city == 'Córdoba'):
        url = 'https://www.meteoblue.com/en/server/search/query3?query={}&itemsPerPage=2'.format(city)
        response = get(url).json()
        lon = response['results'][1].get('lon')
        lat = response['results'][1].get('lat')
        locationCities.append([city, lat, lon])
        continue
    url = 'https://www.meteoblue.com/en/server/search/query3?query={}&itemsPerPage=1'.format(city)
    response = get(url).json()
    lon = response['results'][0].get('lon')
    lat = response['results'][0].get('lat')
    locationCities.append([city, lat, lon])
locationCities = pd.DataFrame.from_records(locationCities, columns=['City', 'Latitude', 'Longitude'])
locationCities.head()

,City,Latitude,Longitude
0,London,51.508499,-0.12574
1,Berlin,52.524399,13.41050
2,Madrid,40.416500,-3.70256
3,Rome,41.891899,12.51130
4,Paris,48.853401,2.34880


Now, we will merge the coordinates with the dataframe

In [6]:
df = pd.merge(df, locationCities, on=['City'], how='inner')

Let's display the firsts rows

In [7]:
df.head()

,City,Country,Population,Latitude,Longitude
0,London,United Kingdom,"8,825,001",51.508499,-0.12574
1,Berlin,Germany,"3,723,914",52.524399,13.41050
2,Madrid,Spain,"3,223,334",40.416500,-3.70256
3,Rome,Italy,"2,863,970",41.891899,12.51130
4,Paris,France,"2,140,526",48.853401,2.34880


In this point of the process, we can display the map with the different cities around Europe:

In [8]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  51.05 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  35.67 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  36.30 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  48.53 MB/s


In [9]:
lat_europe = 50.52596
long_europe = 15.25512

map_europe = folium.Map(location=[lat_europe, long_europe], zoom_start=4)
cities=df
# add markers to map
for lat, lng, city, population in zip(df['Latitude'], df['Longitude'], df['City'], df['Population']):
    label = '{}\n{}'.format(city, population)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_europe)  
    
map_europe

To continue, let's connect with the Foursquare API and download all the data for the different cities

In [10]:
CLIENT_ID = '5IQXGYRYYHEY30DJ4RZQR3QQZIVVMOWUADQN4OHLFS0GJ00V' # your Foursquare ID
CLIENT_SECRET = 'UUE4HG0QTUKHRGQGXXFKRM1SOXQDXVA4S2E0WXYMSSFN3DYR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + '5IQXGYRYYHEY30DJ4RZQR3QQZIVVMOWUADQN4OHLFS0GJ00V')
print('CLIENT_SECRET:' + 'UUE4HG0QTUKHRGQGXXFKRM1SOXQDXVA4S2E0WXYMSSFN3DYR')

Your credentails:
CLIENT_ID: 5IQXGYRYYHEY30DJ4RZQR3QQZIVVMOWUADQN4OHLFS0GJ00V
CLIENT_SECRET:UUE4HG0QTUKHRGQGXXFKRM1SOXQDXVA4S2E0WXYMSSFN3DYR


Test extracting main venues from the first city, in this case will be London

In [11]:
city_latitude = df.loc[0, 'Latitude'] # neighbourhood latitude value
city_longitude = df.loc[0, 'Longitude'] # neighbourhood longitude value

city_name = df.loc[0, 'City'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(city_name, 
                                                               city_latitude, 
                                                               city_longitude))

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 10000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    city_latitude, 
    city_longitude, 
    radius, 
    LIMIT)
url # display URL

Latitude and longitude values of London are 51.508499, -0.12574.


'https://api.foursquare.com/v2/venues/explore?&client_id=5IQXGYRYYHEY30DJ4RZQR3QQZIVVMOWUADQN4OHLFS0GJ00V&client_secret=UUE4HG0QTUKHRGQGXXFKRM1SOXQDXVA4S2E0WXYMSSFN3DYR&v=20180605&ll=51.508499,-0.12574&radius=10000&limit=100'

In [13]:
results = get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c78fe85351e3d13a8054ffa'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4ac518cdf964a520e6a520e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/museum_art_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d18f941735',
         'name': 'Art Museum',
         'pluralName': 'Art Museums',
         'primary': True,
         'shortName': 'Art Museum'}],
       'id': '4ac518cdf964a520e6a520e3',
       'location': {'address': 'Trafalgar Sq',
        'cc': 'GB',
        'city': 'London',
        'country': 'United Kingdom',
        'distance': 194,
        'formattedAddress': ['Trafalgar Sq',
         'London',
         'Greater London',
         'WC2N 5DN',
         'United Kingdom'],
        'labeledLatLn

In [14]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
venues = results['response']['groups'][0]['items']
    
city_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
city_venues =city_venues.loc[:, filtered_columns]

# filter the category for each row
city_venues['venue.categories'] = city_venues.apply(get_category_type, axis=1)

# clean columns
city_venues.columns = [col.split(".")[-1] for col in city_venues.columns]

city_venues.head()

,name,categories,lat,lng
0,National Gallery,Art Museum,51.508876,-0.128478
1,Gordon's Wine Bar,Wine Bar,51.507895,-0.123232
2,Trafalgar Square,Plaza,51.508048,-0.127699
3,Corinthia Hotel,Hotel,51.506607,-0.124460
4,National Portrait Gallery,Art Gallery,51.509438,-0.128032


Define function to get venues for the city

In [22]:
def getCityVenues(names, latitudes, longitudes, radius=10000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [23]:
europe_venues = getCityVenues(names=df['City'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

London
Berlin
Madrid
Rome
Paris
Bucharest
Vienna
Hamburg
Warsaw
Budapest
Barcelona
Munich
Milan
Prague
Sofia
Brussels
Birmingham
Cologne
Naples
Stockholm
Turin
Marseille
Amsterdam
Zagreb
Valencia
Leeds
Copenhagen
Kraków
Frankfurt
Riga
Seville
Łódź
Palermo
Zaragoza
Athens
Rotterdam
Helsinki
Wrocław
Stuttgart
Glasgow
Düsseldorf
Dortmund
Genoa
Essen
Vilnius
Sheffield
Leipzig
Málaga
Gothenburg
Bremen
Dublin
Lisbon
Dresden
Manchester
Poznań
The Hague
Bradford
Hanover
Antwerp
Lyon
Nuremberg
Edinburgh
Duisburg
Liverpool
Toulouse
Gdańsk
Bristol
Tallinn
Murcia
Bratislava
Palma de Mallorca
Szczecin
Bologna
Florence
Brno
Las Palmas
Plovdiv
Bochum
Iași
Cardiff
Bydgoszcz
Coventry
Utrecht
Wuppertal
Leicester
Nice
Bilbao
Lublin
Belfast
Wakefield
Aarhus
Varna
Bielefeld
Timișoara
Malmö
Alicante
Córdoba
Nottingham
Bari
Wigan
Thessaloniki
Bonn
Cluj-Napoca
Constanța
Catania
Münster
Karlsruhe
Craiova
Mannheim
Galați


In [25]:
print(europe_venues.shape)
europe_venues.head()

(10829, 7)


,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,London,51.508499,-0.12574,National Gallery,51.508876,-0.128478,Art Museum
1,London,51.508499,-0.12574,Gordon's Wine Bar,51.507895,-0.123232,Wine Bar
2,London,51.508499,-0.12574,Trafalgar Square,51.508048,-0.127699,Plaza
3,London,51.508499,-0.12574,Corinthia Hotel,51.506607,-0.124460,Hotel
4,London,51.508499,-0.12574,National Portrait Gallery,51.509438,-0.128032,Art Gallery


In [27]:
print('There are {} uniques categories.'.format(len(europe_venues['Venue Category'].unique())))

There are 406 uniques categories.


Let's analyze each city by the categories of the venues

In [30]:
# one hot encoding
europe_onehot = pd.get_dummies(europe_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
europe_onehot['City'] = europe_venues['City'] 

# move neighborhood column to the first column
fixed_columns = [europe_onehot.columns[-1]] + list(europe_onehot.columns[:-1])
europe_onehot = europe_onehot[fixed_columns]

europe_onehot.head()

,City,Accessories Store,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,...,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,London,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,London,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,London,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,London,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,London,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
europe_onehot.shape

(10829, 407)

In [33]:
europe_grouped = europe_onehot.groupby('City').mean().reset_index()
europe_grouped.head()

,City,Accessories Store,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,...,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Aarhus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.03,0.0,0.0,0.0,0.00,0.00,0.0,0.0
1,Alicante,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.01,0.0,0.0,0.0,0.00,0.00,0.0,0.0
2,Amsterdam,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.01,0.00,0.0,0.0,0.0,0.01,0.02,0.0,0.0
3,Antwerp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.01,0.0,0.0,0.0,0.02,0.00,0.0,0.0
4,Athens,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.01,0.04,0.0,0.0,0.0,0.00,0.00,0.0,0.0


In [34]:
europe_grouped.shape

(110, 407)

In [45]:
num_top_venues = 5

for city in europe_grouped['City']:
    print("----"+city+"----")
    temp = europe_grouped[europe_grouped['City'] == city].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aarhus----
               venue  freq
0               Café  0.11
1        Coffee Shop  0.06
2  French Restaurant  0.05
3               Park  0.05
4                Bar  0.04


----Alicante----
                venue  freq
0    Tapas Restaurant  0.13
1          Restaurant  0.07
2  Spanish Restaurant  0.06
3               Plaza  0.06
4         Coffee Shop  0.05


----Amsterdam----
               venue  freq
0              Hotel  0.08
1               Café  0.04
2         Restaurant  0.04
3     Sandwich Place  0.04
4  French Restaurant  0.04


----Antwerp----
                venue  freq
0         Coffee Shop  0.11
1        Cocktail Bar  0.05
2      Clothing Store  0.05
3   French Restaurant  0.04
4  Italian Restaurant  0.03


----Athens----
           venue  freq
0  Historic Site  0.09
1            Bar  0.07
2           Café  0.07
3    Coffee Shop  0.06
4   Cocktail Bar  0.06


----Barcelona----
                venue  freq
0               Hotel  0.13
1    Tapas Restaurant  0.09
2        

                 venue  freq
0                 Café  0.12
1          Coffee Shop  0.06
2  Romanian Restaurant  0.05
3                 Park  0.04
4         Dessert Shop  0.03


----Karlsruhe----
                venue  freq
0                Café  0.06
1  Italian Restaurant  0.05
2                 Pub  0.04
3               Plaza  0.04
4         Coffee Shop  0.04


----Kraków----
               venue  freq
0               Café  0.11
1              Hotel  0.06
2                Bar  0.05
3  Polish Restaurant  0.05
4              Plaza  0.04


----Las Palmas----
                venue  freq
0          Restaurant  0.09
1  Spanish Restaurant  0.08
2    Tapas Restaurant  0.07
3                 Bar  0.06
4               Plaza  0.05


----Leeds----
             venue  freq
0              Bar  0.13
1              Pub  0.12
2      Coffee Shop  0.10
3             Café  0.06
4  Thai Restaurant  0.06


----Leicester----
               venue  freq
0                Pub  0.13
1        Coffee Shop  0.10
2  

            venue  freq
0           Plaza  0.10
1  Ice Cream Shop  0.09
2            Café  0.07
3   Historic Site  0.06
4           Hotel  0.06


----Utrecht----
                 venue  freq
0           Restaurant  0.07
1          Coffee Shop  0.07
2                  Bar  0.06
3         Burger Joint  0.04
4  Indie Movie Theater  0.03


----Valencia----
                venue  freq
0  Italian Restaurant  0.06
1               Hotel  0.05
2  Spanish Restaurant  0.04
3    Tapas Restaurant  0.04
4               Plaza  0.04


----Varna----
                  venue  freq
0                  Café  0.06
1                   Bar  0.06
2                 Hotel  0.04
3  Gym / Fitness Center  0.04
4                 Plaza  0.04


----Vienna----
                 venue  freq
0                Hotel  0.10
1                Plaza  0.09
2           Restaurant  0.08
3                 Café  0.07
4  Austrian Restaurant  0.06


----Vilnius----
            venue  freq
0            Café  0.07
1           Hotel  0.07


In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [48]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
cities_venues_sorted = pd.DataFrame(columns=columns)
cities_venues_sorted['City'] = europe_grouped['City']

for ind in np.arange(europe_grouped.shape[0]):
    cities_venues_sorted.iloc[ind, 1:] = return_most_common_venues(europe_grouped.iloc[ind, :], num_top_venues)

cities_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aarhus,Café,Coffee Shop,Park,French Restaurant,Bakery,Bar,Scandinavian Restaurant,Hotel,Wine Bar,Music Venue
1,Alicante,Tapas Restaurant,Restaurant,Plaza,Spanish Restaurant,Hotel,Coffee Shop,Pizza Place,Burger Joint,Bookstore,Ice Cream Shop
2,Amsterdam,Hotel,Café,Restaurant,Coffee Shop,Cocktail Bar,Bookstore,Sandwich Place,French Restaurant,Burger Joint,Beer Bar
3,Antwerp,Coffee Shop,Cocktail Bar,Clothing Store,French Restaurant,Sandwich Place,Spanish Restaurant,Plaza,Bar,Juice Bar,Italian Restaurant
4,Athens,Historic Site,Café,Bar,Cocktail Bar,Coffee Shop,Greek Restaurant,Wine Bar,Theater,Meze Restaurant,Bookstore
